In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ### Gang Green All Time Stats
# Chris McAllister
# 
# #### This notebook gathers all the Oakland Hockey Stats for all 4 GG teams, for every season that which the website has data.
# 
# #### Oultine:
# 
# ##### 1) Import libraries
# ##### 2) Establish mapping of GG Team ID's in URL to Team Names (Gang Green 1, 2, etc.)
# ##### 3) Read in a CSV that converts SeasonIDs to the Season Name
# ##### 4) Establish an empty dataset of only the column header. We later append our data to this empty dataframe.
# ##### 5) For each GG team, loop over all the season that they have data on going back to 2007.
# ##### 6) Light data manipulation. Removing columns, create Points per Game metric, etc.
# ##### 7) Join dataset back to Team ID and Season ID tables so we can know what year / team name the IDs correspond to. 

# 1) Import libraries. All we need is Pandas
import pandas as pd

# Ignore any warning messages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:



# 2) Create simple mapping table that converts the Team ID to a Team Name.
def create_team_dim_table(team_ids, team_names):
    """
    Arguments:
    team_ids (list): a list of team ids. These numbers are how the website defines our team. For example, Gang Green2 has a team id #692 on the stats website. 
    team_names (list): a list of team names in the same order as the team ids

    This function zips the two lists together and returns a dataframe that we will later use to create our final dataset. 
    """

    team_dim_df = pd.DataFrame(zip(gg_team_ids, gg_team_names), columns = ['TeamID', 'Team Name'])
    return team_dim_df

GG_team_ids = [1843, 692, 4622, 4818]
GG_team_names = ['Gang Green 1', 'Gang Green 2', 'Gang Green 3', 'Gang Green 4']
team_dim = create_team_dim_table(GG_team_ids, GG_team_names)


# 3) Read in our manually supplied mapping table.
season_dim = pd.read_csv('OaklandHockeySeasondDim.csv')

# 4) Establish empty dataframe to get the column names. Next we'll append each team-season combination to this df
team_id = 692
url = 'https://stats.sharksice.timetoscore.com/display-schedule?team='+str(team_id)+'&season=' + str(50) + '&league=27&stat_class=1'

# Establish base data frame so we can later append onto it
df = pd.read_html(url)
df[1].columns = df[1].columns.droplevel()

df_main = df[1].iloc[0:0]

# 5) For each GG team, loop over every season to grab their stats and append it to our big dataframe we made above.
# This is written very inefficiently and will be pretty slow. 
for team_id in gg_team_ids:

    for season_id in season_dim['SeasonID']:
        url = 'https://stats.sharksice.timetoscore.com/display-schedule?team='+str(team_id)+'&season=' + str(season_id) + '&league=27&stat_class=1'

        # There are gaps in season IDs (for example there's no season #34). 
        # This would cause an error when reading the URL so we need to handle that with the try / except code block below. 
        try:
            df = pd.read_html(url)
            season_dim = pd.read_csv('OaklandHockeySeasonDim.csv')
            # Remove first layer of column (that just say says 'Game Results')
            #df[0].columns = df[0].columns.droplevel()
            #df[0].head()

            df[1].columns = df[1].columns.droplevel()
            df[1]['SeasonID'] = int(season_id)
            df[1]['TeamID'] = int(team_id)
            

            df_main = pd.concat([df_main, df[1]])
            #df_main['SeasonID'] = season_id

        except:
            pass


# 6) Data manipulation. Removing certain columns. Tweaking data types. Joing to our dimension tables. 

# Cast as integers so the join below works (otherwise it won't recognize 5.0 as 5, etc.)
df_main['SeasonID'] = df_main['SeasonID'].astype(int) 
df_main['TeamID'] = df_main['TeamID'].astype(int) 

# Convert season IDs (#40) to Season Name (Fall 2017)
df_final = pd.merge(left = df_main, right = season_dim, how = 'left', left_on = 'SeasonID', right_on = 'SeasonID')

# Only select necessarry columns
col = ['Name', '#', 'GP', 'Goals', 'Ass.', 'PPG', 'PPA', 'SHG', 'SHA', 'GWG',
       'GWA', 'PSG', 'ENG',  'PIMs', 'Hat', 'Pts', 'SeasonID', 'Season', 'Year', 'TeamID',
       'SeasonName']

df_final = df_final[col]

# Create a GPG and Pts per game metric. 
df_final['GPG'] = df_final['Goals'] / df_final['GP']
df_final['Pts_PG'] = df_final['Pts'] / df_final['GP']

df_final['SeasonID'] = df_final['SeasonID'].astype(int) 
df_final['TeamID'] = df_final['TeamID'].astype(int)


# Get team name from Team ID (GG 1, 3, etc.)
df_final_2 = pd.merge(left = df_final, right = team_dim, how = 'left', left_on = 'TeamID', right_on = 'TeamID')

# Ouput results to CSV
df_final_2.to_csv('OaklandHockeyData.csv', index = False)
